<center> <h1> Projet Méthodologique : Comparaison des methodes de récalage </h1></center>



.

# Importation des librairies

In [1]:
from __future__ import print_function 
from __future__ import division  
import numpy as np
import math
from statistics import mean 
import matplotlib.pyplot as plt
import os
import cv2 as cv 
import sys
import cv2
from PIL import Image
from numpy import asarray
#from skimage.measure import structural_similarity as ssim
from skimage import measure
from skimage.metrics import structural_similarity as ssim
import time
import tarfile
import xml.dom.minidom as md
import skimage as ski # Scikit-Image : Image processing
import skimage.feature
import skimage.morphology
from skimage import img_as_float
from skimage.metrics import mean_squared_error as mse  #from skimage.measure import compare_mse as mse

import scipy.spatial.distance as ssd # Distance metric
from sklearn.cluster import KMeans
# For Coherent Point Drift (CPD) registration :
import pycpd
from pycpd import rigid_registration # CPD
from functools import partial
from socket import socket
# For mutual information registration :
from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import (RigidTransform2D, AffineTransform2D)
from dipy.viz import regtools
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)

from pycpd import RigidRegistration
from PIL import Image as im 
#! pip install scikit-image
#! pip install dipy
#! pip install scipy
#! pip install pycpd
#! pip install skimage 
#! pip install cv2
#! pip install scikitlearn

C:\ProgramData\Anaconda3\lib\site-packages\dipy\viz\__init__.py:31: UserWarning: You do not have FURY installed. Some visualization functions might not work for you. For installation instructions, please visit: https://fury.gl/
  warnings.warn(


# I. Organisation des images

    On repartit les images de chaque feuille dans un dossier afin de faciliter le recalage et l'enregistrement de l'image         récalée.

In [148]:
n=178
for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Documents\2020-07-23 Croiss lésion 7dpi-ROI')
    image1=cv2.imread('image1 ({}).jpg'.format(i))
    
    os.chdir (r'C:\Users\Adama\Documents\2020-07-27 Croiss lésions 11dpi-ROI')
    image2=cv2.imread('image2 ({}).jpg'.format(i))
    
    os.chdir (r'C:\Users\Adama\Documents\2020-07-29 Croiss lésions 13dpi-ROI')
    image3=cv2.imread('image3 ({}).jpg'.format(i))
    
    #Enregistrement des images
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    cv2.imwrite("Image (1).jpg",image1)
    cv2.imwrite("Image (2).jpg",image2)
    cv2.imwrite("Image (3).jpg",image3)

# II. Fonctions utiles

## 1. Fonctions pour la comparaison des méthodes 

On appelle la fonction RMSE (Root Mean Square Error) et SSIM (Structural Similarity Index) dans une seule fonction qu'on utilisera pour comparer les performances des 3 methodes de récalage

In [2]:
def comparer_images(imageA, imageB):
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB,multichannel=True,gaussian_weights=True)
    return m,s

## 2. Les Fonctions de recalage

### 2.1 Recalage par la methode ORB

In [9]:
MAX = 700 #Maximum de caracteristiques de l'image
GOOD_MATCH_PERCENT = 0.50

def recalageORB(im1, im2):

    # Conversion en gris
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
  
    # Detection des caracteristiques ORB et calcul des descripteurs
    orb = cv2.ORB_create(MAX)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  
    # Correspondances des caracteristiques
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
  
     # Ordonner les caracteristiques par score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Effacer les mauvaises correspondances
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("correspondanceI_J.jpg", imMatches)
  
    # Extraction des bonnes correspondances
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
  
     #homographie
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
    height, width, channels = im2.shape
    imagerecale = cv2.warpPerspective(im1, h, (width, height))
  
    return imagerecale, h


### 2.2 Recalage par la methode de l'information mutuelle

Dans la théorie, l'information mutuelle est définie comme suit :

$$
I(X;Y) = \sum_{y \in Y} \sum_{x \in X}
                 p(x,y) \log{ \left(\frac{p(x,y)}{p(x)\,p(y)}
                              \right) }
$$

Ici $X$ et $Y$ répresentent nos deux images.

In [10]:
def MutualInfo(image1,image2) :
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    MI : Mutual Information method for affine registration and recalibration \n
    img1_o = Original first Image or Visible Image (in grayscale) \n
    img2_o = Original second Image or Fluorescent Image (in grayscale) \n
    Multimodal = logical, True, if the images are from different modalities \n
    Multistart = logical, True, if you want to test several initial conditions \n
    Sym = logical, True if you want to test symmetry with additional initial conditions \n
    Affine = if True, performs an affine transformation after the rigid one (with scaling and skew) \n
    border = size of the border to add to avoid bad cropping \n
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    # Load the data
    img1 = ski.color.rgb2gray(image1)
    img2 = ski.color.rgb2gray(image1)
    # Extract masks
    img1_m = img1 <1 # img1 mask
    img2_m = img2 < 1 # img2 mask
    if np.sum(img1_m) == 0 :
        print('Erreur : cette image ne contient pas de pixels !')
        return None    
    # Affine registration with Mutual Information criterion (by default)
    nbins = 32
    sampling_prop = None
    metric = MutualInformationMetric(nbins, sampling_prop)
    affreg = AffineRegistration(metric=metric)

    # Rigid registration  (only rotation and translation)
    rigid = affreg.optimize(img1,img2,RigidTransform2D(),None)
    img2_new = rigid.transform(img2) # Recalibrated image
        
    # Affine registration (rotation, translation, scaling and skew), if wanted
    affine = affreg.optimize(img1,img2,AffineTransform2D(),None,starting_affine=rigid.affine)
    img2_new = affine.transform(img2_new)
    img2_new[img2_new == 0] = 1
    # Output
    return img2_new

### 2.3 Recalage par la methode Coherent Point Drift

In [11]:
def coherentpointdrift(img1_o,img2_o):
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    CPD : Coherent Point Drift method for Point registration and recalibration \n
    img1_o = Original first Image or Visible Image \n
    img2_o = Original second Image or Fluorescent Image \n
    Multimodal = logical, True, if the images are from different modalities (e.g. visible and fluorescent) \n
    Multistart = logical, True, if you want to test several initial conditions \n
    Sym = logical, True if you want to test symmetry with additional initial conditions \n
    Param = logical, True if you want to retrieve the transformation matrix and the position used for registration \n
    border = size of the border to add to avoid bad cropping \n
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
   # Load the data
    img1 = np.copy(img1_o)
    img2 = np.copy(img2_o)

    # Extract masks
    img1_m = ski.color.rgb2gray(img1) < 1 # img1 mask
    img2_m = ski.color.rgb2gray(img2) <1 # img2 mask

    # Extract contours with Canny filter
    img1_c = ski.feature.canny(img1_m) # Contour from first image
    img1_p = np.transpose(np.where(img1_c))  # Location of the contour points
    img2_c = ski.feature.canny(img2_m) # Contour from second image
    img2_p = np.transpose(np.where(img2_c))  # Location of the contour points
    
    # Callback function
    def trace(iteration, error,X,Y): # Function to see the progress of the registration
        print('Iteration: {:d}\nError: {:06.4f}'.format(iteration, error))
    callback = partial(trace) # Follow the progress of the rigid registration function
   
    # Background value
    background =[255,255,255]

   # If you don't use Multistart
    #reg =RigidRegistration(**{ 'X': img1_p,'Y':img2_p,'max_iterations':200, 'tolerance':0.0001} )
    reg = RigidRegistration(**{'X': X, 'Y': Y})
    reg.register(callback) # Rigid registration
    
    # Recalibration of every coordinates
    T = np.float32(np.c_[reg.R,[reg.t[1],reg.t[0]]]*reg.s) # Transformation matrix
    img2_new = cv2.warpAffine(img2,T,dsize=(img1.shape[1],img1.shape[0]),borderValue=background)

    # Output 
    return img2_new

# III. Application : Recalage et Calcul des erreurs

## 2. Avec les 2 methodes : ORB, MI

## La méthode ORB

In [7]:
n=35
mseorb=[]
ssimorb=[]
tempsorb=[]


for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    image1=cv2.imread('Image (1).jpg') #image de reference
    image2=cv2.imread('Image (2).jpg') #image traitée
   
    #Appel des fonctions de recalage : ORB, MI
    t0 = time.time()
    imagerecaleorb,h=recalageORB(image2, image1)
    t1 = time.time()
    
    # CALCUL DES ERREURS
    
    #MSE et SSIM
    mseorbi,simorbi =comparer_images(imagerecaleorb, image2)
    
    mseorb.append(mseorbi)
    ssimorb.append(simorbi)
    
    
    # Calcul du TEMPS d'execution
    tempsorb.append(t1-t0)
        
    #Enregistrement des images recalées convertie
    cv.imwrite("imagerecaleORB{}.jpg".format(i), imagerecaleorb)
    
#Affichage ORB
msemoyorb=mean(mseorb)
ssimmoyorb=mean(ssimorb)
tempsmoyorb=mean(tempsorb)

print('RMSE moyenne pour ORB : ', math.sqrt(msemoyorb))
print('SSIM moyenne pour ORB : ' ,ssimmoyorb)
print('Temps moyenne dexecution pour ORB : ' ,tempsmoyorb)


RMSE moyenne pour ORB :  117.88192846670168
SSIM moyenne pour ORB :  0.7688497737332084
Temps moyenne dexecution pour ORB :  2.818543945040022


# La méthode MI

In [ ]:
n=35
tempsmi=[]

for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    image1=cv2.imread('Image (1).jpg') #image de reference
    image2=cv2.imread('Image (2).jpg') #image traitée
   
    #Appel des fonctions de recalage : ORB, MI
    t1 = time.time()
    imagerecalemi=MutualInfo(image2,image1)
    t2= time.time()

    # Calcul du TEMPS d'execution
    tempsmi.append(t2-t1)
        
    #Enregistrement des images recalées convertie
    imagerecalemi=cv2.normalize(imagerecalemi, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    cv.imwrite("imagerecaleMI{}.jpg".format(i), imagerecalemi)
    

In [ ]:
n=35
msemi=[]
ssimmi=[]

for i in range(1,n+1):
    os.chdir (r'C:\Users\Adama\Desktop\Images_Plantes\feuille ({})'.format(i))
    imagerecalemi=cv2.imread("imagerecaleMI{}.jpg".format(i),0) #image de reference
    image2=cv2.imread('Image (2).jpg',0) #image traitée
    #MSE et SSIM 
    msemii,simmii=comparer_images(imagerecalemi, image2)
    msemi.append(msemii)
    ssimmi.append(simmii)

#Affichage MI
msemoymi=mean(msemi)
ssimmoymi=mean(ssimmi)
tempsmoymi=mean(tempsmi)


In [88]:
print('RMSE moyenne pour MI : ' ,math.sqrt(msemoymi))
print('SSIM moyenne pour MI : ' ,ssimmoymi)
print('Temps moyenne dexecution pour MI : ', tempsmoymi)

RMSE moyenne pour MI :  51.017722778461994
SSIM moyenne pour MI :  0.9981027855373399
Temps moyenne dexecution pour MI :  1736.3432385444642
